In [ ]:
import torch
import os
import cv2
import grayscale_moments
import resnet
import hog
import extract_features

In [ ]:
data1_path = '../data/Part1/Part1'
data2_path = '../data/Part2/Part2'

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

In [ ]:
# Create a dictionary to store the results
extracted_features = {
    'file_path': [],
    'class': [],
    'cm': [],
    'hog': [],
    'avgpool': [],
    'layer3': [],
    'fc': [],
}

In [ ]:
extracted_features = []
# For each image in data1_path extract the features
for root, dirs, files in os.walk(data1_path):
    for image in files:
        print(f'Path: {image}')
        features_dict = extract_features.extract_features(image, model)
        # Append the features to the list
        extracted_features.append(features_dict)


In [ ]:
extracted_features = []
# For each image in data1_path extract the features
for root, dirs, files in os.walk(data1_path):
    for image in files:
        if not image.endswith('.jpg'):
            continue

        # Class can be glioma, menin, tumor. Get the class from the filename
        class_name = image.split('_')[0]+'_' + image.split('_')[1]
        image_path = os.path.join(f'{class_name}/', image)
        # Get the full path to the image
        image_path = os.path.abspath(os.path.join(data1_path, image_path))
        print(f'Image Path {image_path}')
        features_dict = extract_features.extract_features(image_path, model)
        
        # Append the features to the list
        extracted_features.append(features_dict)


In [ ]:
# Save the extracted features into a pt file
torch.save(extracted_features, '../data/extracted_features_test.pt')

In [ ]:
# Load the extracted features from the pt file
extracted_features_test = torch.load('../data/extracted_features_test.pt')
# Print the extracted features
print(extracted_features_test)

In [ ]:
# Load the extracted features from the pt file
extracted_features_original = torch.load('../data/extracted_features.pt')
# Print the extracted features
print(extracted_features_original)

In [ ]:
for i in range(len(extracted_features_test)):
    avgpool_test = extracted_features_test[i]['fc']
    avgpool_original = extracted_features_original[i]['fc']
    if not torch.allclose(avgpool_test, avgpool_original, atol=1e-10):
        print(f'Avgpool feature mismatch at index {i}')
        print(f'Test: {avgpool_test}')
        print(f'Original: {avgpool_original}')

In [ ]:
part2_test_features = extract_features.extract_features(f'{data2_path}/brain_glioma/brain_glioma_1962.jpg', model)
avgpool_test = part2_test_features['layer3']

In [ ]:
# Extract the top 10 most similar images from extracted_features_original (for the feature avgpool)
similar_images = []
for i in range(len(extracted_features_original)):
    avgpool = extracted_features_original[i]['layer3']
    # Calculate the cosine similarity between the two features
    similarity = torch.nn.functional.cosine_similarity(avgpool_test, avgpool, dim=0)
    # Append the similarity to the list
    similar_images.append(similarity.item())
# Sort the list of similarities
similar_images = sorted(similar_images, reverse=True)
print(similar_images)